# Porous Media Flow Simulator

## Overview of the simulator



- Discretization : uniform cartesian grid.
- Fluids: single phase and two phase fluid flow.
- Wells : injectors and producers wells.
- Boundary conditions : Dirichlet constant pressure and no-flow bounday conditions.
- Solving Methods : Lagging Coefficients Method (LCM) and Fully Implicit Method (FIM).
- Linear solvers : direct and iterative solver.

This simulator was done as a requirement for the class Advanced Reservoir Simulation. 
Before working on this simulator I used [Matlab Reserovir Simulation Toolbox (MRST)](https://www.sintef.no/projectweb/mrst/ "MRST") for a few weeks, some aspects of this simulator are inspired by the MRST. The idea for the `UnitRegistry` class,  I got it from  [pint](https://pint.readthedocs.io/en/0.6/tutorial.html).

In [11]:
#%matplotlib inline
from modules.simulator.simulator import *
from modules.simulator.two_phase_flow import *
from modules.simulator.simplots import *
from modules.simulator.units import UnitRegistry
import numpy as np
from time import process_time
import os
import time

## Example 1: Single phase flow

### Units and constants
There are two classes to help us manage the units, the **UnitRegistry** and **Constants** We can input the variables values in the International Sytem Units (SI) or in Filed Units, they are converted to SI units. The output of the simulator is also in SI units and then it can be converted to the desired units with the help of the module.

In [7]:
# New units registry
u = UnitRegistry()

distance = 3 * u.meter
print ("Distance is {:.3f} meters".format(distance))

# The units are converted to SI
distance = 3 * u.feet
print ("Distance is {:.3f} meters".format(distance))

# The units can be mixed together 
perm = 1 * u.milli * u.darcy
print("Permeabiliy is {:.3E} Pa-s".format(perm))

Distance is 3.000 meters
Distance is 0.914 meters
Permeabiliy is 9.869E-16 Pa-s


### Grid 

In [8]:
# Create grid
# Number of cells in each direction
Nx, Ny, Nz = np.array([15, 15, 4])
# Dimension of each cell
Sx, Sy, Sz = np.array([30, 30, 30]) * u.feet
# Depth of each cell
depth = np.hstack((Sz * 3 * np.ones([Nx * Ny, ]),
                   Sz * 2 * np.ones([Nx * Ny, ]),
                   Sz  * 1 * np.ones([Nx * Ny, ]),
                   Sz * 0 * np.ones([Nx * Ny, ])))

grid = uniformCartesianGrid(Nx, Ny, Nz, Sx, Sy, Sz, depth)

print("Number of cells {}.".format(grid.cellnumber))
print("Cell size {} meters.".format(grid.cellsize))
print("Cell volume {:.3f} m^3".format(grid.cellvolume))
print("Grid dimensions {}".format(grid.dim))

Number of cells 900.
Cell size [ 9.144  9.144  9.144] meters.
Cell volume 764.555 m^3
Grid dimensions [15 15  4]



### Rock

In [9]:
# Create rock
# Randowm porosiy and permeability fields.
poro = np.random.random_sample(grid.cellnumber)
perm = 10 * np.random.random_sample(grid.cellnumber) * u.milli * u.darcy

# Rock compressibility is equal  and constant for all cells.
cr = 3E-6 / u.psi
# The porosity is a function of rock compressibility
porofunc = lambda p: poro * np.exp(cr * (p - 2800 * u.psi))
rock = Rock(perm, poro, cr, porofunc)

## Fluid
There are two classes to specify a fluid in the simulator, `singleFluid` and `blackOil`. 

- #### Single phase flow
`singleFluid` is for a single phase fluid with constant compressibility. The other properties, viscosity, density and formation volume factor, are expressed as a function of pressure.



In [23]:
#Example for water
cf = 5E-5 / u.psi
# The functions should be in the correct units
miu = lambda p:  1.0 * np.exp(5E-5 * (p/u.psi -2800)) * u.centi * u.poise
rho = lambda p: 1000 * np.exp(cf * (p - 15 * u.psi) )  * u.kilogram/ u.meter**3
fvf = lambda p: 1.0 * np.exp(- cf * (p - 15 * u.psi) )   # adimensional
fluid = singleFluid(miu, rho, fvf, cf)

- #### Two phase flow
The class `blackOil` implements a two phase fluid, one phase being water and the other oil. The class is called `blackOil` even tough it is lacking support for gas flow.

In [13]:
# Example for water and oil flow


## Plotting functions

The `simplots` module has several functions to plot the data that is generated with the simulator. These functions save the images to the specified folder. For example,  `plotCellValueS2D`, plots each individual layer as a subplot.

In [14]:
# Plot rock properties
plotCellValues2D(grid, rock.poro, 'viridis', 
                 np.min(rock.poro), np.max(rock.poro), 
                 title='Porosity', filename = ".\images\poro")

permtoplot = np.log(rock.perm/u.darcy/u.milli)
plotCellValues2D(grid, permtoplot, 'magma', np.min(permtoplot), np.max(permtoplot), 
  title='Permeability [log(mD)]', filename = ".\images\perm")

C:\Users\Jaime\Anaconda3\lib\site-packages\matplotlib\figure.py:1718: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


<img src=".\images\poro.png">
<img src=".\images\perm.png">

## Wells

The simulator only support vertical wells that are located in the center of a cell and completed in one single layer. The producer wells are specified by setting the bottom hole pressure (bhp) and the injector wells by setting the injection rate.
For this example, there is one  injector and two producers.

In [15]:
#W = Wells(grid, rock, fluid)

## Boundary Conditions
By defaul, all the boundaries of the reservoir have a no-flow condition. With the `fuction` we can also set a constant pressure condition in a boundary. Currently, the function ony works with the Lagging Cofficient Method.

## Well schedule
With the well schedule we can set the rate and the pressure of each well.

In [16]:
## Solver

In [17]:
## Visualizing the results


## Example 2: Two phase flow

## Conclusion

## Notes 
This reservoir simulator was developed as a final project for the class Advanced Reservoir Simulation 696.

## References
Gong, X., Gonzalez, R., McVay, D. A., & Hart, J. D. (2014, December 1). Bayesian Probabilistic Decline-Curve Analysis Reliably Quantifies Uncertainty in Shale-Well-Production Forecasts. Society of Petroleum Engineers. doi:10.2118/147588-PA


In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# Using the style sheet found here  Lorena Barba /* https://github.com/barbagroup/CFDPython/blob/master/styles/custom.css */